In [1]:
# n = Name of the excel file
# s = Sheet Name within the excel file
# r = Range of data which needs to be imported
# d = Data object that needs to be sent while exporting back

def import_excel(n,s,r):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    data = sheet.range(r).value
    return data 


def export_excel(n,s,r,d):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    sheet.range(r).value = d

In [2]:
#dt = Date on which operation is to be performed
#cal1 = Country of Calendar 1, For USA pass argument US
#cal2 = Country of Calendar 2, For India pass argument IN .. This is an optional argument
# At present this function suports a max of 2 calendars only for MFBD

def mfbd(dt,cal1, cal2="NIL"):
    from datetime import date
    from datetime import timedelta
    import holidays
    direction = 1
    calendar1 = holidays.country_holidays(cal1)
    if cal2 == "NIL":
        while dt.weekday() > 4 or dt in calendar1:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    else:
        calendar2 = holidays.country_holidays(cal2)
        while dt.weekday() > 4 or dt in calendar1 or dt in calendar2:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    return dt

In [3]:
def volsurf(data,fcycode):
    import pandas as pd
    import numpy as np
    import holidays
    from datetime import date,timedelta
    import plotly.graph_objects as go
    import nbformat
    df = pd.DataFrame(data)
    df.columns = ['Tenor','Unit','ATM','25D RR','25D BF','10D RR','10D BF']
    PricingDate = date.today()
    SDate = mfbd(PricingDate + timedelta(days=2),'US',fcycode) #Calculating the Spot Date for EUR/USD
    #Generate Maturity Dates
    df['Date'] = SDate
    for index,row in df.iterrows():
        if row['Unit']=='d':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(days=row['Tenor']),'US',fcycode)
        elif row['Unit']=='w':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(weeks=row['Tenor']),'US',fcycode)
        elif row['Unit']=='m':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(months=row['Tenor']),'US',fcycode)
        elif row['Unit']=='y':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(years=row['Tenor']),'US',fcycode)
    
    #We need to have some logic to price extreme vols here. I'm choosing flat for now
    #An alternative is to use a multiplier of slope (Say 3 times) to extrapolate as long as vols ho higher than 10D / 90D 
    

    df['Date'] = pd.to_datetime(df['Date'])
    df['1D']= df['ATM'] + df['10D RR']/2 + df['10D BF']
    df['10D']= df['ATM'] + df['10D RR']/2 + df['10D BF']
    df['25D']= df['ATM'] + df['25D RR']/2 + df['25D BF']
    df['50D']= df['ATM']
    df['75D']= df['ATM'] - df['25D RR']/2 + df['25D BF']
    df['90D']= df['ATM'] - df['10D RR']/2 + df['10D BF']
    df['99D']= df['ATM'] - df['10D RR']/2 + df['10D BF']
    
    df = df.drop(columns=['Tenor','Unit','ATM','25D RR','25D BF','10D RR','10D BF']) #Dropping unnecessary columns

    #print(df) #Uncomment this line if you want to see the processed dataframe
    fig = go.Figure(data=[go.Surface(z=df[['10D','25D','50D','75D','90D']].values,
                                     x=['10D','25D','50D','75D','90D'],
                                     y=df['Date'].dt.strftime('%Y-%m-%d').values)])
    fig.update_layout(title='FX Option Volatility Surface', autosize=True,
                        scene = dict(
                            xaxis_title='Delta',
                            yaxis_title='Maturity Date',
                            zaxis_title='Implied Volatility (%)'),
                        )
    #fig.show() #Uncomment this line if you want to see the plot in a separate window
    return df

In [4]:
def bootstrap(data,i=-1,p=1):
    from datetime import date,timedelta
    import holidays
    import pandas as pd
    PricingDate = date.today() #Use todays date as the Pricing Date
    #PricingDate = date(2025,5,19) # For testing currently
    SDate = mfbd(PricingDate + timedelta(days=2),'US') #Calculating the Spot Start Date
    df = pd.DataFrame(data)
    df.columns = ['Tenor','Unit','Rate']
    i = int(i)
    if i>-1:
        df.iat[i,1]=df.iat[i,1]+0.0001*p
        
    df['MatDate'] = SDate 
    df['PmtDate'] = SDate

    #Populate Maturity Date and Payment Date adjusting for pay delay

    for index,row in df.iterrows():
        if row['Unit']=='d':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(days=row['Tenor']),'US')
        elif row['Unit']=='w':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(weeks=row['Tenor']),'US')
        elif row['Unit']=='m':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(months=row['Tenor']),'US')
        elif row['Unit']=='y':
            df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(years=row['Tenor']),'US')
        
        df.at[index,'PmtDate'] = mfbd(df.at[index,'MatDate'] + pd.offsets.DateOffset(days=2),'US') #SOFR Swaps have 2 days Payment Delay
    
    df['MatDate'] = pd.to_datetime(df['MatDate']).dt.date # To remove hh:mm:ss from Date
    df['PmtDate'] = pd.to_datetime(df['PmtDate']).dt.date
    df['DC'] = df['PmtDate'].diff(periods=1) #Compute Periodic day counts
    
    for i in range(0,15):
        df.at[i,'DC']= pd.to_timedelta(df.at[i,'PmtDate']-SDate) #Compute DC value from Start Date for tenors upto 1 year
    
    
    df['DC'] = pd.to_numeric(df['DC'].dt.days)
    df['DC']=df['DC']/360 #Since Daycount is Act/360 for USD SOFR
 
    import numpy as np
    df['DF'] = 1.000000
 
    SDate = mfbd(SDate,'US')


    #Just testing, delete later
    #for index,row in df.iterrows():
    #    df.at[index,'DF'] = 1/(pow(1+row['Rate'],row['Year']))


    #Compute actual Curve DFs for USD SOFR
    #The for loop is massively simplified to solve a linear system of equations. Will write a full document on how this is calculated later. 
    # For queries in the interim email pushkars@myyahoo.com 
    dfdccumprodsum = 0
    for index,row in df.iterrows():
        if index < 14:
            df.at[index,'DF'] = 1/(1+df.at[index,'Rate']*df.at[index,'DC'])
        else:
            df.at[index,'DF'] = (1 - df.at[index,'Rate']*dfdccumprodsum)/(1+df.at[index,'Rate']*df.at[index,'DC'])
            dfdccumprodsum = dfdccumprodsum + df.at[index,'DC']*df.at[index,'DF']

    #Compute Zeros for USD SOFR
    # Please note that zeros requires an assumption of the approach. Here we are going to try and match
    # it with that of bloomberg by assuming continuous compounding
    # Different systems implement this bit in a different manner


    df['Zero']=1.000000
    
    for index,row in df.iterrows():
        #p = pd.to_timedelta(df.at[index,'PmtDate']-df.at[index,'MatDate'])
        #t = float(p.days)/365 + df.at[index,'Year']
        t = (df.at[index,'PmtDate'] - SDate).days/365 #Note this implementation is slightly different and less accurate as its 
        #dividing by 365 to give years which will create issues especially as we cross leap years. Will update in future iterations
        #print(t)
        df.at[index,'Zero'] = np.log(1/df.at[index,'DF'])/t

    #Add 1 as Discount Factor for Spot Date in the Discount Factors Data Frame for US Rates 
    df.loc[-1] = [0.0,"w",0.0,SDate,SDate,0,1.00000,0]
    df.index = df.index+1
    df = df.sort_index()
    
        
    #print(df)
    return df

In [11]:
def bootstrapfx(data,usdf,fcycode,divfactor=10000,i=-1,p=1):
    import pandas as pd
    import numpy as np
    from datetime import date,timedelta, datetime, time
    import math 
    df=pd.DataFrame(data)
    df.columns=['Tenor','Unit','Bid','Ask']
    spotrate= (df.at[0,'Bid'] + df.at[0,'Ask'])/2
    #df = df.drop(df.index[0]) #Drop the spot rate row
    #df = df.reset_index(drop=True) #Reset index after dropping spot rate row
   
    df['OR']=spotrate + (df['Bid'] + df['Ask'])/ (2*divfactor) #Allows flexibility to use 100 for JPY and 10,000 by default for others
    df.at[0,'OR']=spotrate  #Set the outright for spot rate row
    #Set Pricing Date
    PDate = date.today() #You can set it to desired pricing date
    SDate = mfbd(PDate + timedelta(days=2),'US',fcycode) #Calculating the Spot Start Date basis code recieved 
    
    #Generate Maturity Dates
    df['Date'] = SDate
    for index,row in df.iterrows():
        if row['Unit']=='d':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(days=row['Tenor']),'US',fcycode)
        elif row['Unit']=='w':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(weeks=row['Tenor']),'US',fcycode)
        elif row['Unit']=='m':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(months=row['Tenor']),'US',fcycode)
        elif row['Unit']=='y':
            df.at[index,'Date'] = mfbd(row['Date'] + pd.offsets.DateOffset(years=row['Tenor']),'US',fcycode)


    #This will require us to interpolate and we are creating a float column as we cant interpolate on dates directly
    MyTime = time(0,0,0)
    ref = pd.Timestamp.combine(SDate,MyTime)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Days']=(df['Date']-ref)/timedelta(days=1)
    usdf['PmtDate'] = usdf['PmtDate'].astype("datetime64[ns]")
    usdf['Days']=(usdf['PmtDate']-datetime.combine(SDate,MyTime))/timedelta(days=1)
    #print(usdf)
    df['USDF']=np.interp(df['Days'],usdf['Days'],usdf['DF'])
    if  fcycode == 'XECB' or fcycode == 'GB' or fcycode == 'AU':
        df['FCYDF']=df['OR']*df['USDF']/spotrate
    else:
        df['FCYDF']=df['USDF']*spotrate/df['OR']    
    
    df['FCYYld']=(1/df['FCYDF']-1)*365/df['Days']
    
    df.at[0,'FCYYld'] = 0.0 #Override first yield with 0 as spot has no yield
    
    #This step overwrites the DFs and the Outrights with perturbed values if any
    #i = int(i)
    #if i>-1:
    #    df.iat[i,6]=df.iat[i,6]+0.0001*p
    #df['FCYDF']=1/(1+df['FCYYld']*df['Days']/365)
    #df['OR']=spotrate *df['USDF']/df['FCYDF']

    return df

In [16]:
import pandas as pd
from datetime import date,timedelta
import holidays
data = import_excel("Market Data.xlsx","FXO","b3:h17")
eursurf=volsurf(data,"XECB") #XECB is the code for TARGET Calendar
data = import_excel('Market Data.xlsx','FXO','K3:M21')
usdf=bootstrap(data)
data = import_excel('Market Data.xlsx','FXO','C32:F54')
eurdf=bootstrapfx(data,usdf,'XECB') #XECB is the code for TARGET Calendar


#print(eurdf)
#print(eursurf)

#We now have both the vol surface and the discount factors for EUR. We can now price options using these inputs

#We will now implement Black Scholes pricing for FX Options using the vol surface and discount factors generated above
#Supporting only EUR/USD options for now. will extend later to other currencies 


This volLookup function is the most important piece of FX Option pricing and often not well understood. We tend to think of parmeters entered and the vols automatically generated in Bloomberg / other systems but there is a nuanced calculation behind this which is not so intuitive. 

I will try and explain a bit about it here 

To get the right vols given a strike we need to know the right Delta (Since the vols are quoted in delta terms)
However, the formula for Delta is N(d1) which has vols as an input in the formula
So this is an optimization problem which starts with a guess and then requires a numerical method to solve

If you need further clarity or would like to discuss further please email pushkars@myyahoo.com and i'll try and revert as soon as possible

Do also note that this implementation is not perfect but a close approximation

FX Vols upto 1Y are typically ATMF but after that they are traded as DNS and DNS K does not equal ATMF K 

Actually DNS K has delta = 0.5 or 50% and ATMF is close to 50% 

For FXO Delta = N(D1) .. by the identity N(0)=0.5 you can equate D1 = 0 and figure it out easily

In [18]:
import pandas as pd
import numpy as np
from datetime import date,timedelta
import holidays
from scipy.stats import norm
def volLookup(volsurf,dfs,style,K,D,Spot): #K is strike and D is delivery date
    guess = 50 #Rough initial guess for Delta @ 50%
    #This obtains the outright forward rate for the given delivery date
    F=np.interp(pd.to_datetime(D).value,dfs['Date'].values.astype('datetime64[ns]').astype(np.int64),dfs['OR'].values)
    T= (D-pd.Timestamp(Spot)).days/365
    #print(T)
    if style == 'Call':
        #Convergence should be achieved in 3-5 iterations normally
        for i in range(5):
            #Step 1 - Get vol for the delta estimate
            vol=interp_vol(volsurf,D,guess)
            #Step 2 - Get Delta for the vol estimate
            guess = norm.cdf((np.log(F/K)+(0.5*vol*vol*T))/(vol*np.sqrt(T)))*100
            #print("Iteration ",i+1,": Vol = ",vol,", Delta = ",guess) #Uncomment this to see vol iterations
    else:
        #This is a Put instead of a call now 
        for i in range(5):
            #Step 1 - Get vol for the delta estimate
            vol=interp_vol(volsurf,D,100-guess) #This is because 25D Call = 75 Delta Put 
            #Step 2 - Get Delta for the vol estimate
            guess = abs((norm.cdf((np.log(F/K)+(0.5*vol*vol*T))/(vol*np.sqrt(T)))-1)*100)
            #print("Iteration ",i+1,": Vol = ",vol,", Delta = ",guess) #Uncomment this to see vol iterations
    
    #Lets just price the option as well using the Garman-Kohlhagen model
    d1= (np.log(F/K)+(0.5*vol*vol*T))/(vol*np.sqrt(T))
    d2 = d1 - (vol*np.sqrt(T))                                
    rd = np.interp(pd.to_datetime(D).value,dfs['Date'].values.astype('datetime64[ns]').astype(np.int64),dfs['USDF'].values)
    #Note rd above is not the interest rate but the USD discount factor itself , logic will have to be changed if pair is not eur or gbp
    if style == 'Call':
        # Price a call option
        px = rd * (F*norm.cdf(d1)-K*norm.cdf(d2))       
    else:
        #Price a put option
        px = rd *(K*norm.cdf(-d2)-F*norm.cdf(-d1))
    #Return the converged volatility
    #print(vol," ",px," ",guess)
    return F,vol,px,guess
            

def interp_vol(df, target_date, target_delta):
    
    # Ensure Date is datetime
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    # Delta columns (convert "1D"→1, "10D"→10, ..., "99D"→99)
    delta_cols = [c for c in df.columns if c.endswith('D') and c != 'Date']
    delta_values = np.array([int(c.replace('D','')) for c in delta_cols])
    
    # Interpolate each column on Date
    # gives a single interpolated row for the exact date
    df_date_indexed = df.set_index('Date')
    interp_row = df_date_indexed.reindex(
        df_date_indexed.index.union([target_date])
    ).sort_index().interpolate(method='time').loc[target_date, delta_cols].values

    #Interpolate along Delta axis
    final_vol = np.interp(target_delta, delta_values, interp_row)

    return final_vol

In [20]:
data = import_excel('Market Data.xlsx','FXOTrades','b2:h3') #Change to H3 later to price multiple trades
fxoTrades=pd.DataFrame(data)
fxoTrades.columns = ['Pair','Not CCY','Notional','Style','B/S','Strike','Delivery']
fxoTrades['Vol']=0.0 #Initialize Dummy values for Vol
fxoTrades['Delta']=0.0 #Initialize Dummy values for Vol
fxoTrades['Px']=0.0 #Initialize Dummy values for Px
#print(fxoTrades)
for index,row in fxoTrades.iterrows():
    if row['Pair']=='EUR/USD':
        SDate = mfbd(date.today() + timedelta(days=2),'US','XECB') #Using a fixed pricing date for testing
        F,vol,px,delta=volLookup(eursurf,eurdf,row['Style'],row['Strike'],row['Delivery'],SDate)
        fxoTrades.loc[index,'Fwd']=F
        fxoTrades.loc[index,'Vol']=vol
        fxoTrades.loc[index,'Delta']=delta
        fxoTrades.loc[index,'Px']=px

print(fxoTrades)
        

      Pair Not CCY   Notional Style  B/S  Strike   Delivery       Vol  \
0  EUR/USD     EUR  1000000.0   Put  Buy   1.160 2026-01-15  0.056581   
1  EUR/USD     EUR  2000000.0  Call  Buy   1.165 2026-09-25  0.063468   

       Delta        Px       Fwd  
0  43.747855  0.007819  1.163507  
1  58.022673  0.032221  1.176719  


![alternative text](./FXO1.png)
![alternative text](./FXO2.png)

The numbers are fairly close Please note that i had to type in market data manually and then price in bloomberg which is why there is a difference. This gap will almost vanish if i align the data but that defeats the whole purpose of replication